In [67]:
# MLP - with using paper Bengio et al.2003

In [68]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [69]:
# read in all the words
words = open('cleaned_names.txt', 'r').read().splitlines()
words[:8]

['aaban', 'aabid', 'aabidah', 'aabir', 'aabriella', 'aada', 'aadam', 'aadarsh']

In [70]:
len(words)

29681

In [71]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(set(''.join(words)))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [72]:
# build the dataset

block_size = 3 # context length: how many characters to use to predict the next one?
X, Y = [], []
for w in words: # [:5]

  # print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    # print(''.join([itos[i] for i in context]), '---->', itos[ix])
    context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [73]:
X.shape, X.dtype, Y.shape, Y.dtype 

(torch.Size([212725, 3]), torch.int64, torch.Size([212725]), torch.int64)

In [ ]:
# Converting the above code into a function

def build_dataset(words):
  block_size = 3 # context length: how many characters to use to predict the next one
  X, Y = [], []
  for w in words:

    # print(w)
    context = [0] * block_size
    for ch in w + '.':
      X.append(context)
      Y.append(ix)
      # print(''.join([itos[i] for i in context]), '---->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1]) # 80%
Xdev, Ydev = build_dataset(words[n1:n2]) # 10%
Xte, Yte = build_dataset(words[n2:]) # 10%

torch.Size([170379, 3]) torch.Size([170379])
torch.Size([21222, 3]) torch.Size([21222])
torch.Size([21124, 3]) torch.Size([21124])


In [75]:
len(words)

29681

In [76]:
n1

23744

In [77]:
n2

26712

In [78]:
n2 - n1

2968

In [79]:
len(words) - n2

2969

In [8]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1,  1],
        ...,
        [26, 26, 25],
        [26, 25, 26],
        [25, 26, 24]])

In [9]:
Y # labels

tensor([ 1,  1,  2,  ..., 26, 24,  0])

In [10]:
C = torch.randn(27, 2) # each 27 characters is represented by a 2D embedding

In [11]:
C

tensor([[-0.3105, -1.0320],
        [ 0.7077, -0.9061],
        [ 0.7283, -1.0139],
        [ 1.2915, -0.0988],
        [-2.0374,  0.3136],
        [-0.3543,  0.8327],
        [-1.0881,  0.6891],
        [ 0.4127,  0.3940],
        [-0.2155,  2.7478],
        [-1.2722, -0.6875],
        [-1.3414,  0.5701],
        [-0.1373, -0.3903],
        [-1.6822,  0.0451],
        [ 0.5717,  0.3583],
        [-1.1429,  0.4985],
        [-0.3768, -1.0261],
        [-1.0329, -0.0399],
        [-1.0006, -0.0970],
        [-1.8061, -0.8585],
        [ 1.3643, -1.0987],
        [-2.4586,  0.1586],
        [-0.2568,  0.5095],
        [-0.1178, -2.2129],
        [ 1.1398,  0.6049],
        [-0.3578, -1.8244],
        [ 1.4668,  1.7589],
        [ 1.3276,  0.1486]])

In [12]:
C[5]

tensor([-0.3543,  0.8327])

In [13]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C # one hot vector

tensor([-0.3543,  0.8327])

In [14]:
C[[5,6,7]] # indexing with list

tensor([[-0.3543,  0.8327],
        [-1.0881,  0.6891],
        [ 0.4127,  0.3940]])

In [15]:
C[torch.tensor([5,6,7])] # indexing with tensor

tensor([[-0.3543,  0.8327],
        [-1.0881,  0.6891],
        [ 0.4127,  0.3940]])

In [16]:
C[X].shape

torch.Size([212725, 3, 2])

In [17]:
X[13,2] # 13th example, 2nd character

tensor(1)

In [18]:
C[X[13,2]] # embedding of the 13th example, 2nd character

tensor([ 0.7077, -0.9061])

In [19]:
C[1]

tensor([ 0.7077, -0.9061])

In [20]:
emb = C[X]
emb.shape

torch.Size([212725, 3, 2])

In [21]:
# Constructing the hidden layer
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [22]:
h = emb.view(-1, 6) @ W1 + b1

In [23]:
h

tensor([[-0.8068, -4.9106,  2.3838,  ..., -1.7078, -0.1546,  3.0983],
        [-1.0960, -5.2463,  2.6028,  ..., -0.3570,  0.3323,  1.1896],
        [-0.6553, -5.1618,  2.3500,  ...,  0.9404,  0.6533, -0.1856],
        ...,
        [ 1.0929, -0.4905, -1.3371,  ...,  6.7747, -1.1854, -0.1631],
        [ 3.2253,  1.0637, -5.7640,  ...,  5.8431, -0.7389, -6.3776],
        [ 0.9707, -0.4101, -1.4959,  ..., -0.7585, -1.2331,  1.9368]])

In [24]:
# torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape # first, second, third characters of the first example concatenated

In [25]:
# torch.cat(torch.unbind(emb, 1), 1).shape # split the tensor along the second dimension

In [26]:
a = torch.arange(18)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [27]:
a.shape

torch.Size([18])

In [28]:
a.view(2, 9) # view as 2x9 matrix

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17]])

In [29]:
a.view(9, 2)

tensor([[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15],
        [16, 17]])

In [30]:
a.view(3, 3, 2)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]]])

In [31]:
a.storage() # memory address

C:\Users\DC\AppData\Local\Temp\ipykernel_11896\3903475333.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  a.storage() # memory address


 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]

In [32]:
emb.shape

torch.Size([212725, 3, 2])

In [33]:
# emb.view(32, 6) == torch.cat(torch.unbind(emb, 1), 1)

In [34]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [35]:
h

tensor([[-0.6678, -0.9999,  0.9831,  ..., -0.9364, -0.1534,  0.9959],
        [-0.7990, -0.9999,  0.9891,  ..., -0.3426,  0.3206,  0.8305],
        [-0.5752, -0.9999,  0.9820,  ...,  0.7354,  0.5739, -0.1835],
        ...,
        [ 0.7979, -0.4546, -0.8710,  ...,  1.0000, -0.8291, -0.1617],
        [ 0.9968,  0.7871, -1.0000,  ...,  1.0000, -0.6285, -1.0000],
        [ 0.7490, -0.3886, -0.9044,  ..., -0.6402, -0.8435,  0.9593]])

In [36]:
h.shape

torch.Size([212725, 100])

In [37]:
(emb.view(-1, 6) @ W1).shape

torch.Size([212725, 100])

In [38]:
b1.shape

torch.Size([100])

In [39]:
# 32, 100
# broadcasting to 100
# 1 , 100

In [40]:
# Creating the final layer (output layer)
W2 = torch.randn(100, 27)
b2 = torch.randn(27)

In [41]:
logits = h @ W2 + b2

In [42]:
logits.shape

torch.Size([212725, 27])

In [43]:
counts = logits.exp()

In [44]:
prob = counts / counts.sum(-1, keepdim=True)

In [45]:
prob.shape

torch.Size([212725, 27])

In [46]:
prob

tensor([[4.6411e-12, 3.7023e-06, 7.7571e-10,  ..., 3.7635e-05, 2.8628e-09,
         8.6724e-07],
        [9.3649e-13, 3.2718e-09, 5.0918e-15,  ..., 1.0854e-13, 4.4865e-10,
         3.5606e-14],
        [2.6611e-09, 3.6588e-11, 3.0789e-11,  ..., 1.6573e-11, 6.4346e-13,
         2.9550e-11],
        ...,
        [5.1188e-04, 2.3226e-10, 9.1284e-04,  ..., 1.6173e-07, 5.4173e-10,
         8.2909e-06],
        [9.7287e-04, 2.1630e-11, 4.0519e-03,  ..., 1.7025e-07, 1.1054e-09,
         7.5274e-10],
        [6.8683e-06, 1.5538e-05, 8.5361e-02,  ..., 7.2883e-06, 2.4015e-10,
         4.9454e-05]])

In [47]:
prob[0].sum() # normalized

tensor(1.)

In [48]:
Y # labels

tensor([ 1,  1,  2,  ..., 26, 24,  0])

In [49]:
torch.arange(36)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])

In [50]:
# prob[torch.arange(36), Y] # probabilities of the true characters

In [51]:
# loss = -prob[torch.arange(36), Y].log().mean() # cross entropy loss
# loss

In [52]:
# ----------------- Summary -----------------

In [126]:
Xtr.shape, Ytr.shape

(torch.Size([170379, 3]), torch.Size([170379]))

In [128]:
g = torch.Generator().manual_seed(2147483647) # seed for reproducibility
C = torch.randn(27, 2, generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn(100, 27, generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [129]:
sum(p.nelement() for p in parameters) # total number of parameters

3481

In [130]:
# Enabling gradient computation
for p in parameters:
  p.requires_grad = True

In [131]:
# finding out the learning rate
lre = torch.linspace(-3, 0, 1000) # 1000 points between 0.001 and 1
lrs = 10 ** lre # learning rates
# lrs

In [133]:
lri = []
lossi = []

for i in range(10000): # iterations

# mini-batch construction
  ix = torch.randint(0, Xtr.shape[0], (36,)) # 36 random indices from the dataset
# Forward pass

  emb = C[Xtr[ix]] # embedding 
  h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # hidden layer
  logits = h @ W2 + b2 # output layer
# counts = logits.exp() # unnormalized probabilities
# prob = counts / counts.sum(1, keepdim=True) # normalized probabilities
# loss = -prob[torch.arange(36), Y].log().mean() # cross entropy loss
#  but never use the above code in practice as it is not efficient and numerically unstable
  loss = F.cross_entropy(logits, Ytr[ix]) # cross entropy loss
  # print(loss.item())


  # Backward pass
  for p in parameters:
    p.grad = None

  loss.backward()

  # Update
  # lr = lrs[i]
  lr = 0.01
  for p in parameters:
    p.data += -lr * p.grad # learning rate e.g. -1 (lr)

  # track stats
  # lri.append(lre[i])
  # lossi.append(loss.item())  

In [134]:
print(loss.item())  

2.6821733627002686e-05


In [119]:
# plt.plot(lri, lossi)
# plt.xscale('log')

In [120]:
# F.cross_entropy(logits, Y) # PyTorch's implementation of cross entropy we can put the cross entropy directly in this code with the help of Pytorch directly in this code which will be more efficient for forward pass and backward pass and numerical stability.

In [121]:
logits.max(1) # max in python is a function that returns two values: the maximum value and its index

torch.return_types.max(
values=tensor([32.7060, 32.7060, 32.7060, 32.7060, 32.7060, 32.7060, 32.7060, 32.7060,
        32.7060, 32.7060, 32.7060, 32.7060, 32.7060, 32.7060, 32.7060, 32.7060,
        32.7060, 32.7060, 32.7060, 32.7060, 32.7060, 32.7060, 32.7060, 32.7060,
        32.7060, 32.7060, 32.7060, 32.7060, 32.7060, 32.7060, 32.7060, 32.7060,
        32.7060, 32.7060, 32.7060, 32.7060], grad_fn=<MaxBackward0>),
indices=tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [122]:
Y # labels

tensor([ 1,  1,  2,  ..., 26, 24,  0])

In [123]:
# creating minibatches
torch.randint(0, X.shape[0], (36,)) # random integers between 0 and 4

tensor([200942, 193377,  15713,   1918,  25474, 104182, 175136,  91081,  43629,
        112115,  52083, 126974,  97342, 202003,  47761,  78811, 169877,  71965,
        111059,  67682,  76013,  53080,   6155,   7684,  25702,  87683,  18999,
         49521,  85345, 178685,  91047, 162382, 188496,  50814, 159152,  30669])

In [135]:
# evaluation

emb = C[Xdev] # embedding 
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # hidden layer
logits = h @ W2 + b2 # output layer
loss = F.cross_entropy(logits, Ydev) # cross entropy loss
loss


tensor(2.6822e-05, grad_fn=<NllLossBackward0>)

In [136]:
# loss on training set
emb = C[Xtr] # embedding
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # hidden layer
logits = h @ W2 + b2 # output layer
loss = F.cross_entropy(logits, Ytr) # cross entropy loss
loss

tensor(2.6822e-05, grad_fn=<NllLossBackward0>)

In [125]:
# Training split, dev / validation split, test split
# Training split: 80% of the data
# Dev split: 10% of the data
# Test split: 10% of the data